<a href="https://colab.research.google.com/github/Cpalm03/Big-Data-Analytics/blob/main/ICP7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
TF_USE_LEGACY_KERAS=1

In [2]:
!pip install scikeras

In [21]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
import re
from sklearn.preprocessing import LabelEncoder

# Load and preprocess the data
data = pd.read_csv('drive/MyDrive/Colab Notebooks/DataSets/Sentiment.csv')
data = data[['text', 'sentiment']]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply(lambda x: re.sub('[^a-zA-Z0-9\s]', '', x))

# Tokenization
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

# Define model creation function
def createmodel(embed_dim=128, lstm_out=196):
    model = Sequential()
    model.add(Embedding(input_dim=max_features, output_dim=embed_dim, input_length=X.shape[1]))
    model.add(Dense(256, activation="sigmoid"))
    model.add(LSTM(lstm_out, dropout=0.2))  # Removed recurrent_dropout
    model.add(Dense(3, activation='softmax'))  # Output layer
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Encode labels
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)

# Split the data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Wrap the model in KerasClassifier for GridSearchCV
model = KerasClassifier(model=createmodel, verbose=0)

# Define the parameter grid for grid search
param_grid = {
    'batch_size': [32, 64],
    'epochs': [2]
}

# Initialize GridSearchCV with the model and parameter grid
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=1, error_score = 'raise')

# Fit GridSearchCV
grid_result = grid.fit(X_train, Y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Save the best model
best_model = grid_result.best_estimator_.model
best_model.save('sentiment.h5')


Fitting 3 folds for each of 2 candidates, totalling 6 fits


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


ValueError: Sequential model 'sequential_43' has no defined outputs yet.

In [ ]:
from keras.models import load_model
model1 = load_model('sentiment.h5')
x = tokenizer.texts_to_sequences(["A lot of good things are happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump"])
x = pad_sequences(x, maxlen=28)
model1.predict(x)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step


array([[0.49588552, 0.20981357, 0.29430088]], dtype=float32)